<a href="https://colab.research.google.com/github/samuelcueva/Data-Glacier-Internship/blob/master/week%206/Data_Ingestion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data Ingestion

## Dependencies

In [1]:
!python -m pip install "dask[complete]"

     |████████████████████████████████| 769 kB 13.5 MB/s 
     |████████████████████████████████| 118 kB 76.2 MB/s 
     |████████████████████████████████| 766 kB 83.7 MB/s 
     |████████████████████████████████| 1.0 MB 61.4 MB/s 
     |████████████████████████████████| 722 kB 64.2 MB/s 
     |████████████████████████████████| 722 kB 30.3 MB/s 
     |████████████████████████████████| 715 kB 70.2 MB/s 
     |████████████████████████████████| 705 kB 65.0 MB/s 
     |████████████████████████████████| 699 kB 59.3 MB/s 
     |████████████████████████████████| 696 kB 59.2 MB/s 
     |████████████████████████████████| 684 kB 63.4 MB/s 
     |████████████████████████████████| 679 kB 56.8 MB/s 
     |████████████████████████████████| 675 kB 55.0 MB/s 
     |████████████████████████████████| 675 kB 54.3 MB/s 
     |████████████████████████████████| 672 kB 51.0 MB/s 
     |████████████████████████████████| 671 kB 55.3 MB/s 
     |████████████████████████████████| 669 kB 54.4 MB/s 
     |████████

In [2]:
import tensorflow as tf
import pandas as pd
import numpy as np
import dask.dataframe as dd
import os

## Data
* Dataset(Kaggle) : https://www.kaggle.com/kentonnlp/2014-new-york-city-taxi-trips?select=nyc_taxi_data_2014.csv
* The Dataset was download on my Google Drive to be used in this Colab Notebook
* Shared file on Drive: https://drive.google.com/file/d/12HilR722L7wiL4T1iuB83oBlMzrKOSbh/view?usp=sharing


In [3]:
# to use file from Google Drive 
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
# To extract the zio file
!unzip  /content/drive/MyDrive/nyc_taxi_data_2014.csv.zip

Archive:  /content/drive/MyDrive/nyc_taxi_data_2014.csv.zip
  inflating: nyc_taxi_data_2014.csv  


In [5]:
!ls -lh

total 2.4G
drwx------ 5 root root 4.0K Aug  9 04:35 drive
-rw-r--r-- 1 root root 2.4G Sep 20  2019 nyc_taxi_data_2014.csv
drwxr-xr-x 1 root root 4.0K Jul 16 13:20 sample_data


file weigtht: 2.4G 

## Testing reading options 

### Using Pandas

In [6]:
file_path = 'nyc_taxi_data_2014.csv'

In [7]:
%%time
df=pd.read_csv(file_path)
df

<string>:2: DtypeWarning: Columns (8) have mixed types.Specify dtype option on import or set low_memory=False.


CPU times: user 25.8 s, sys: 2.68 s, total: 28.5 s
Wall time: 32.7 s


In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14999999 entries, 0 to 14999998
Data columns (total 18 columns):
 #   Column              Dtype  
---  ------              -----  
 0   vendor_id           object 
 1   pickup_datetime     object 
 2   dropoff_datetime    object 
 3   passenger_count     int64  
 4   trip_distance       float64
 5   pickup_longitude    float64
 6   pickup_latitude     float64
 7   rate_code           int64  
 8   store_and_fwd_flag  object 
 9   dropoff_longitude   float64
 10  dropoff_latitude    float64
 11  payment_type        object 
 12  fare_amount         float64
 13  surcharge           float64
 14  mta_tax             float64
 15  tip_amount          float64
 16  tolls_amount        float64
 17  total_amount        float64
dtypes: float64(11), int64(2), object(5)
memory usage: 2.0+ GB


Using Pandas my local machine reduces its performance because pd.read_csv loads the whole file into memory and my local memory is only 6GB

### Using Dask

Dask first creates a task graph 

In [9]:
df=dd.read_csv(file_path,blocksize=250e6)
df

,vendor_id,pickup_datetime,dropoff_datetime,passenger_count,trip_distance,pickup_longitude,pickup_latitude,rate_code,store_and_fwd_flag,dropoff_longitude,dropoff_latitude,payment_type,fare_amount,surcharge,mta_tax,tip_amount,tolls_amount,total_amount
npartitions=11,,,,,,,,,,,,,,,,,,
,object,object,object,int64,float64,float64,float64,int64,object,float64,float64,object,float64,float64,float64,float64,float64,float64
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...


Now, it's executed

In [10]:
%%time 
df.compute()

/usr/local/lib/python3.7/dist-packages/dask/core.py:121: DtypeWarning: Columns (8) have mixed types.Specify dtype option on import or set low_memory=False.
  return func(*(_execute_task(a, cache) for a in args))


CPU times: user 31.1 s, sys: 1.59 s, total: 32.7 s
Wall time: 1min 4s


,vendor_id,pickup_datetime,dropoff_datetime,passenger_count,trip_distance,pickup_longitude,pickup_latitude,rate_code,store_and_fwd_flag,dropoff_longitude,dropoff_latitude,payment_type,fare_amount,surcharge,mta_tax,tip_amount,tolls_amount,total_amount
0,CMT,2014-01-09 20:45:25,2014-01-09 20:52:31,1,0.70,-73.994770,40.736828,1,N,-73.982227,40.731790,CRD,6.5,0.5,0.5,1.40,0.00,8.90
1,CMT,2014-01-09 20:46:12,2014-01-09 20:55:12,1,1.40,-73.982392,40.773382,1,N,-73.960449,40.763995,CRD,8.5,0.5,0.5,1.90,0.00,11.40
2,CMT,2014-01-09 20:44:47,2014-01-09 20:59:46,2,2.30,-73.988570,40.739406,1,N,-73.986626,40.765217,CRD,11.5,0.5,0.5,1.50,0.00,14.00
3,CMT,2014-01-09 20:44:57,2014-01-09 20:51:40,1,1.70,-73.960213,40.770464,1,N,-73.979863,40.777050,CRD,7.5,0.5,0.5,1.70,0.00,10.20
4,CMT,2014-01-09 20:47:09,2014-01-09 20:53:32,1,0.90,-73.995371,40.717248,1,N,-73.984367,40.720524,CRD,6.0,0.5,0.5,1.75,0.00,8.75
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
186563,VTS,2014-02-03 19:28:00,2014-02-03 19:37:00,5,1.36,-73.991223,40.718037,1,NaN,-74.000675,40.725737,CRD,7.5,1.0,0.5,1.00,0.00,10.00
186564,VTS,2014-02-07 00:55:00,2014-02-07 01:15:00,1,4.26,-73.990247,40.737442,1,NaN,-73.991287,40.692535,CRD,17.5,0.5,0.5,2.00,0.00,20.50
186565,VTS,2014-02-03 18:08:00,2014-02-03 19:29:00,1,57.84,-73.789527,40.645007,5,NaN,-73.776505,40.740790,CSH,153.0,0.0,0.0,0.00,7.08,160.08
186566,VTS,2014-02-07 00:58:00,2014-02-07 01:12:00,1,3.40,-73.983495,40.694153,1,NaN,-74.005953,40.710922,CRD,14.0,0.5,0.5,2.90,0.00,17.90


Using Dask the total processing time is similar, but since Dask only loads a chunk of the Dataframe into memory and the rest is kept on disk until processed, Dask allows me loading large files even though my machine is low on memory.

### Using tf.data API (chosen option)

The Dataset API allows you to build an asynchronous, highly optimized data pipeline. A Tensorflow dataset object is a Python iterable and when operations are executed, this object works as a generator by loading only parts of the dataset into memory.

tf.data.experimental.make_csv_dataset reads CSV files into a dataset, where each element of the dataset is a (features, labels) tuple that corresponds to a batch of CSV rows

# Data Ingestion

Normal reading process of the file with tf.data

In [11]:

nyc_taxi_data_ds = tf.data.experimental.make_csv_dataset(
    file_path,
    batch_size=10, 
    label_name=None,
    num_epochs=1,
    shuffle= False,
    ignore_errors=True,)


viewing one batch

In [12]:
%%time
next(nyc_taxi_data_ds.__iter__())

CPU times: user 41 ms, sys: 12 ms, total: 53.1 ms
Wall time: 231 ms


OrderedDict([('vendor_id', <tf.Tensor: shape=(10,), dtype=string, numpy=
              array([b'CMT', b'CMT', b'CMT', b'CMT', b'CMT', b'CMT', b'CMT', b'CMT',
                     b'CMT', b'CMT'], dtype=object)>),
             ('pickup_datetime', <tf.Tensor: shape=(10,), dtype=string, numpy=
              array([b'2014-01-09 20:45:25', b'2014-01-09 20:46:12',
                     b'2014-01-09 20:44:47', b'2014-01-09 20:44:57',
                     b'2014-01-09 20:47:09', b'2014-01-09 20:45:07',
                     b'2014-01-09 20:44:04', b'2014-01-09 20:43:23',
                     b'2014-01-09 20:43:04', b'2014-01-09 20:50:23'], dtype=object)>),
             ('dropoff_datetime', <tf.Tensor: shape=(10,), dtype=string, numpy=
              array([b'2014-01-09 20:52:31', b'2014-01-09 20:55:12',
                     b'2014-01-09 20:59:46', b'2014-01-09 20:51:40',
                     b'2014-01-09 20:53:32', b'2014-01-09 20:51:01',
                     b'2014-01-09 21:05:45', b'2014-01-09 

The total processing time is similar to Dask, but tf.data takes less memory and allows me to crate a full data pipeline, tha's why it was chosen

In [13]:
column_names = list(nyc_taxi_data_ds.element_spec.keys())
column_names

['vendor_id',
 'pickup_datetime',
 'dropoff_datetime',
 'passenger_count',
 'trip_distance',
 'pickup_longitude',
 'pickup_latitude',
 'rate_code',
 'store_and_fwd_flag',
 'dropoff_longitude',
 'dropoff_latitude',
 'payment_type',
 'fare_amount',
 'surcharge',
 'mta_tax',
 'tip_amount',
 'tolls_amount',
 'total_amount']

In [14]:
%%writefile utils.py
import re
import yaml
import logging
import tensorflow as tf
from csv import reader
import numpy as np
import os



def read_config_file(filepath):
    with open(filepath, 'r') as stream:
        try:
            return yaml.safe_load(stream)
        except yaml.YAMLError as exc:
            logging.error(exc)


def get_csv_header(file_path):
    with open(file_path, 'r') as read_obj:
        csv_reader = reader(read_obj)
        header = next(iter(csv_reader))
    return header


def replacer(string, char):
    pattern = char + '{2,}'
    string = re.sub(pattern, char, string) 
    return string

def header_cleaning(columns):
    columns = map(lambda x: x.lower(), columns)
    columns = map(lambda x: re.sub('[^\w]', '_', x), columns)
    columns = map(lambda x: x.strip('_'), columns)
    columns = map(lambda x: replacer(x,'_'), columns)
    return list(columns)

def validate(file_path,table_config):

    column_names = get_csv_header(file_path)
    #column_names = list(dataset.element_spec.keys())
    column_names = header_cleaning(column_names)
    column_names.sort()
    
    column_expected = table_config['columns']
    sort_expected_column=column_expected.copy()
    sort_expected_column.sort()
    
    if len(column_names) == len(column_expected) and column_names == sort_expected_column:
        print("validation passed")
        val_dataset = tf.data.experimental.make_csv_dataset(
            file_path,
            header=True,
            column_names=column_expected,
            batch_size=100,
            field_delim=table_config['inbound_delimiter'], 
            label_name=None,
            num_epochs=1,
            shuffle=False,
            ignore_errors=True,)
        
        return val_dataset

    else: 
        print("\ncolumn name and column length validation failed")
        mismatched_columns_file = list(set(column_names ).difference(sort_expected_column))
        if len(mismatched_columns_file)!=0:
            print("\nFollowing File columns are not in the YAML file",mismatched_columns_file) 
        missing_YAML_file = list(set(sort_expected_column).difference(column_names))
        if len(missing_YAML_file)!=0:
            print("\nFollowing YAML columns are not in the file uploaded",missing_YAML_file)
        
        return None


def to_string(x,delimiter):
    
    if isinstance(x, bytes):
        return x.decode() + delimiter
    else:
        return str(x) + delimiter


def count(dataset):
    s=0
    for batch in dataset:
        for key,values in batch.items(): 
            s+=len(values)
            break
    return s


def batch_to_write(batch):
    keys = []
    for n,(key,value) in enumerate(batch.items()):
        keys.append(key)

        if n != 0:
            values=np.vstack((values,value.numpy()))
        
        else:
            values = value.numpy()

    return keys,values.T 


def write_txt(txt_file,batch,delimiter):

    header, values = batch_to_write(batch)

    with open(txt_file,'a') as file:
        n_cols = len(header)
        delimiter=list(delimiter*(n_cols-1))
        delimiter.append('\n')
                
        if os.path.getsize(txt_file)==0:
            file.writelines(map(to_string,header,delimiter))
                    
        for row in values:
            line=list(map(to_string,list(row),delimiter))      
            file.writelines(line)
                    
    return n_cols



Writing utils.py


Write YAML file

In [15]:
%%writefile config.yaml
file_type: csv
dataset_name: "2014 New York City Taxi Trips"
file_name: nyc_taxi_data_2014
inbound_delimiter: ","
outbound_delimiter: "|"
skip_leading_rows: 1
columns: 
    - vendor_id
    - pickup_datetime
    - dropoff_datetime
    - passenger_count
    - trip_distance
    - pickup_longitude
    - pickup_latitude
    - rate_code
    - store_and_fwd_flag
    - dropoff_longitude
    - dropoff_latitude
    - payment_type
    - fare_amount
    - surcharge
    - mta_tax
    - tip_amount
    - tolls_amount
    - total_amount

Writing config.yaml


### Read the file using config file

### Write validated file

In [16]:
import utils

def data_ingestion(yaml_file,destination_file):

    config_data = utils.read_config_file(yaml_file)
    file_path = './'+config_data['file_name']+'.'+config_data['file_type']
    
    print('Configuration Data\n',config_data)
    
    dataset = utils.validate(file_path,config_data)

    if dataset != None:

        total_rows=utils.count(dataset)

        for batch in dataset:
            total_cols = utils.write_txt(destination_file,batch,config_data['outbound_delimiter'])
        
        file_size = (os.path.getsize(destination_file))/1e+9

        summary_out_file = {'Total_rows': total_rows,
                            'Total_columns':total_cols,
                            'size_file':str(round(file_size, ndigits=2))+" GB"}

        print('Done! ingestion finished')

        return dataset, summary_out_file

    else:
        return 'Dataset was not created'

Data ingestion

In [17]:
config_file = 'config.yaml'
destination_txt = 'output.txt'
dataset, summary_out_file = data_ingestion(config_file,destination_txt)

Configuration Data
 {'file_type': 'csv', 'dataset_name': '2014 New York City Taxi Trips', 'file_name': 'nyc_taxi_data_2014', 'inbound_delimiter': ',', 'outbound_delimiter': '|', 'skip_leading_rows': 1, 'columns': ['vendor_id', 'pickup_datetime', 'dropoff_datetime', 'passenger_count', 'trip_distance', 'pickup_longitude', 'pickup_latitude', 'rate_code', 'store_and_fwd_flag', 'dropoff_longitude', 'dropoff_latitude', 'payment_type', 'fare_amount', 'surcharge', 'mta_tax', 'tip_amount', 'tolls_amount', 'total_amount']}
validation passed
Done! ingestion finished


In [18]:
summary_out_file

{'Total_columns': 18, 'Total_rows': 14999999, 'size_file': '2.66 GB'}

In [19]:
# dataset batch
next(dataset.__iter__())

OrderedDict([('vendor_id', <tf.Tensor: shape=(100,), dtype=string, numpy=
              array([b'CMT', b'CMT', b'CMT', b'CMT', b'CMT', b'CMT', b'CMT', b'CMT',
                     b'CMT', b'CMT', b'CMT', b'CMT', b'CMT', b'CMT', b'CMT', b'CMT',
                     b'CMT', b'CMT', b'CMT', b'CMT', b'CMT', b'CMT', b'CMT', b'CMT',
                     b'CMT', b'CMT', b'CMT', b'CMT', b'CMT', b'CMT', b'CMT', b'CMT',
                     b'CMT', b'CMT', b'CMT', b'CMT', b'CMT', b'CMT', b'CMT', b'CMT',
                     b'CMT', b'CMT', b'CMT', b'CMT', b'CMT', b'CMT', b'CMT', b'CMT',
                     b'CMT', b'CMT', b'CMT', b'CMT', b'CMT', b'CMT', b'CMT', b'CMT',
                     b'CMT', b'CMT', b'CMT', b'CMT', b'CMT', b'CMT', b'CMT', b'CMT',
                     b'CMT', b'CMT', b'CMT', b'CMT', b'CMT', b'CMT', b'CMT', b'CMT',
                     b'CMT', b'CMT', b'CMT', b'CMT', b'CMT', b'CMT', b'CMT', b'CMT',
                     b'CMT', b'CMT', b'CMT', b'CMT', b'CMT', b'CMT', b'CMT',

### Testing a file that does not match 

Create a new test configuration file by renaming only the csv file

In [20]:
%%writefile test_config.yaml

file_type: csv
dataset_name: "Test dataset"
file_name: test_file
inbound_delimiter: ","
outbound_delimiter: "|"
skip_leading_rows: 1
columns: 
    - vendor_id
    - pickup_datetime
    - dropoff_datetime
    - passenger_count
    - trip_distance
    - pickup_longitude
    - pickup_latitude
    - rate_code
    - store_and_fwd_flag
    - dropoff_longitude
    - dropoff_latitude
    - payment_type
    - fare_amount
    - surcharge
    - mta_tax
    - tip_amount
    - tolls_amount
    - total_amount

Writing test_config.yaml


Create a new csv file with a different header

In [21]:
import csv  

header = utils.get_csv_header(file_path)
header.append('New field')

with open('test_file.csv', 'w') as f:
    writer = csv.writer(f)
    # write the header
    writer.writerow(header)


Validate new file

In [22]:
test_file = './test.txt'
test_dataset = data_ingestion('./test_config.yaml',test_file)

Configuration Data
 {'file_type': 'csv', 'dataset_name': 'Test dataset', 'file_name': 'test_file', 'inbound_delimiter': ',', 'outbound_delimiter': '|', 'skip_leading_rows': 1, 'columns': ['vendor_id', 'pickup_datetime', 'dropoff_datetime', 'passenger_count', 'trip_distance', 'pickup_longitude', 'pickup_latitude', 'rate_code', 'store_and_fwd_flag', 'dropoff_longitude', 'dropoff_latitude', 'payment_type', 'fare_amount', 'surcharge', 'mta_tax', 'tip_amount', 'tolls_amount', 'total_amount']}

column name and column length validation failed

Following File columns are not in the YAML file ['new_field']
